In [1]:
import math
import random
import numpy as np
import pandas as pd
from sklearn import  linear_model
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
from sklearn.linear_model import RidgeCV

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
sales['floors'] = sales['floors'].astype(int) 

In [3]:
def get_numpy_data(data, features, output):

    #Adding a constant collumn with value 1 in the dataframe.
    data['constant'] = 1    
    #Adding the name of the constant collumn in the feature list.
    features = ['constant'] + features
    #Creating Feature matrix(Selecting columns and coverting to matrix).
    features_matrix=data[features].as_matrix()
    
    #Target collumn is converted to the numpy array
    output_array=np.array(data[output])
    
    return(features_matrix.astype(int), output_array.astype(int))


In [4]:
def predict_outcome(feature_matrix, weights):
    weights=np.array(weights)
    predictions = np.dot(feature_matrix, weights)
    
    return(predictions)


### Normalize features

In [12]:
#to normalize features: 
#divide each feature by its 2-norm so that the transformed feature has norm 1.
def normalize_features(features):
    
    norms = np.linalg.norm(features, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    
    normalized_features = features / norms
    
    return (normalized_features, norms)

In [13]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))


[  5.  10.  15.]
[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]


In [14]:
l1_penalty_values = [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]

In [324]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [325]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

[  1.47013605e+02   3.34257264e+05   5.14075870e+02]
[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]


In [74]:
weights = np.array([1., 4., 1.])

In [323]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    
    
    feature_i = feature_matrix[:,i]
    tmp = feature_i * (output - prediction + np.dot(weights[i],feature_i))
    
    ro_i = tmp.sum()
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    #ro_i = (np.dot([feature_matrix[:,i]],(output - prediction)) + np.dot(weights[i],[feature_matrix[:,i]])).sum()
    
    if i == 0: # intercept -- do not regularize
        
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i



In [315]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights) # make sure it's a numpy array
    
    converged = False
    gradient_sum_squares = 0 
        #while not reached maximum number of iterations:
    while not converged:
         # compute the predictions using your predict_output() function
        
        predictions = predict_outcome(feature_matrix, weights)
        
        
       
        # compute the errors as predictions - output
        errors = (predictions - output)
        
     

        
        
        
        need_loop = False
        for i in range(len(weights)): # loop over each weight
            
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
            # use old_weights_i to compute change in coordinate
            change = weights[i] - old_weights_i
            
        
        #stdout.write("\r%d" % int(gradient_magnitude))
        #stdout.flush()
            if change > tolerance:
                need_loop = True
                
            
            
        if need_loop == False:        
            converged = True
                
       
            
     
    return weights

print lasso_cyclical_coordinate_descent(simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)    

[ 21624998.81906042  63157246.42159398         0.        ]


In [288]:
initial_weights = np.array([0.,0.,0.])
l1_penalty = 1e7
tolerance = 1.0

In [249]:
def RSS_calculation(feature_matrix, weights, output):
    prediction = predict_outcome(feature_matrix, weights)
    
    RSS = ((prediction - output)**2).sum()
    
    return RSS

In [327]:
new_weights = lasso_cyclical_coordinate_descent(simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)
new_weights

array([ 21624998.81906042,  63157246.42159398,         0.        ])

In [330]:
RSS = RSS_calculation(simple_feature_matrix, new_weights, output)
print RSS

1.63049248458e+15
